In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()
csv_path = os.getenv("csv_path")
api_key = os.getenv("NOMIC_API_KEY")
url = os.getenv("base_url")
doc_path = os.getenv("doc_path")
# Set the OpenAI API key environment variable
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [2]:
import pandas as pd

data = pd.read_csv(csv_path)

#preview the csv file
data.head()

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


In [3]:
loader = CSVLoader(file_path=csv_path)
docs = loader.load_and_split()

In [4]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_nomic import NomicEmbeddings

embeddings =  NomicEmbeddings(model="nomic-embed-text-v1.5",)
index = faiss.IndexFlatL2(len(embeddings.embed_query(" ")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [5]:
vector_store.add_documents(documents=docs)

['49d10f56-d6e2-4c78-9f54-7babb4c233ae',
 '0c372c1d-4ab7-49d8-88e1-fa9583ec730d',
 '1e4d61cb-e27b-410c-9b46-bc5d4ebd45f8',
 '0f87f369-8691-4d78-bfd5-a2cbc5d0fd27',
 'e5b135b3-e542-477a-adf3-05d9e980310a',
 'd663e81a-9e1f-425a-bff4-8b8ae07ee857',
 '8c0b85a8-727b-466e-89b6-9ac09164b26f',
 '87634b7c-802b-4185-86b9-9228f02ad8f5',
 '7c43f78a-cb5b-4551-ae63-6b8e1e79de9f',
 'a686b3d5-f94f-40a1-b173-1eb7732b4418',
 '1498da55-a528-47ff-bcd0-81bd8485cfc4',
 '54cb2b11-1d68-4838-8dc6-afcf6c59919f',
 '6173d0ad-c9c7-40d9-9eea-88946761c7df',
 'd917d4f4-25b7-4ad2-b86a-fdf887791627',
 '1d32ab8e-a804-4135-ab88-7327797a445a',
 '95ef27cd-9f46-458b-99d2-d4d052c69bba',
 '5f354b51-4ad0-48df-89d9-fdca97b97550',
 '2f11a3df-c685-43f3-8092-7df60c5337a8',
 '6cdac0d9-313f-4b70-8dfe-0fc267c509b4',
 'bfc87a11-d6ed-4788-8b0c-e4d3b90cb8e1',
 '22e73c02-a817-4461-b14d-d2665e9f05e9',
 '5472f2fb-9605-41cd-b902-5b3cde2011d9',
 '7f54ea30-71cb-4c4a-afe3-fd69dbcbe863',
 '92fb45a8-f778-4e87-a10e-cf848e0479f5',
 '655b5bb9-7282-

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector_store.as_retriever()
llm = ChatOpenAI(
                # model="bartowski/Llama-3.2-3B-Instruct-GGU",
                model = "lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
                base_url=url,
                api_key="lm-studio"
            )
# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. specifically for csv files."
    "you are going to over look the columns of the csv file and answer the question"
    "Use the following pieces of retrieved context to answer "
    "when you asked for sum of something please go through the columns and sum the values"
    "when you asked some question please try to answer the question only. for example if you asked for number please answer with the number"
    "the question. If you don't know the answer, say that you "
    ""
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    
])

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [31]:
answer= rag_chain.invoke({"input": "which company does Colleen Howardr work for?, "})
answer['answer']

'Colleen Howard works for Greer and Sons.'

In [15]:
# Sheryl
answer= rag_chain.invoke({"input": "what is  subscription date  Colleen Howardr?"})
answer['answer']

"Colleen Howard's Subscription Date is 2020-08-19."

In [37]:
answer= rag_chain.invoke({"input": "what is rows and column numbers of file provided"})
answer['answer']

'The row numbers are:\n\n* Index: 57, 40, 42, 23 (assuming the first line is row 1)\n\nThe column numbers are not explicitly mentioned, but based on the data, we can identify some columns:\n\n* Customer Id ( appears in every row)\n* First Name\n* Last Name\n* Company\n* City\n* Country\n* Phone 1 and Phone 2 (separate columns for each phone number)\n* Email\n* Subscription Date\n* Website\n\nPlease note that the exact column numbers are not provided, but I can help you with calculations or questions about specific data if needed.'

In [23]:
import pandas as pd
df_xx = pd.read_csv("E:\RAG_Project\data\customers-100.csv")
# df_xx['Subscription Date'].value_counts()
df_xx['Subscription Date'] = pd.to_datetime(df_xx['Subscription Date'], errors='coerce')

# Filter for rows where 'Subscription Date' is in 2022
df_2022 = df_xx[df_xx['Subscription Date'].dt.year == 2022]

# Count subscriptions in 2022
subscription_count_2022 = df_2022['Subscription Date'].count()
print("Subscriptions in 2022:", subscription_count_2022)

Subscriptions in 2022: 19
